# Swiss rail network exploration

This notebook can load and analyse either:

1. The SBB infrastructure dataset that combines line topology with Didok metadata.
2. The nationwide `schienennetz_2056_de` geodatabase from swisstopo.

Set `DATA_SOURCE` in the next cell to switch between them and rebuild the NetworkX graph, connected-components summary, and derived tables.

In [131]:
from pathlib import Path

import pandas as pd
import geopandas as gpd
import networkx as nx
import pickle


In [132]:
DATA_SOURCE = "sbb"  # options: "sbb" or "swisstopo"
EXPORT_GRAPH = True

BASE_DIR = Path("../datasets/switzerland")
SBB_LINE_PATH = BASE_DIR / "sbb-linie-mit-betriebspunkten.csv"
SBB_STATION_PATH = BASE_DIR / "sbb-dienststellen-gemass-opentransportdataswiss.csv"
SWISSTOPO_GDB_PATH = BASE_DIR / "schienennetz_2056_de.gdb"

GRAPH_OUTPUTS = {
    "sbb": BASE_DIR / "sbb_rail_network.gpickle",
    "swisstopo": BASE_DIR / "swiss_rail_network_swisstopo.gpickle",
}

LEGACY_GRAPH_PATHS = {
    "sbb": BASE_DIR / "swiss_rail_network.gpickle",
}

station_metadata = pd.read_csv(SBB_STATION_PATH, sep=';')
station_metadata['abbreviation_clean'] = (
    station_metadata['abbreviation']
    .astype(str)
    .str.strip()
    .str.upper()
)
stop_point_mask = station_metadata['stopPoint'].astype(str).str.lower() == 'true'
station_abbreviation_set = set(
    station_metadata.loc[stop_point_mask, 'abbreviation_clean'].dropna().tolist()
)
station_abbreviation_set.discard('NAN')

GRAPH_OUTPUT_PATH = GRAPH_OUTPUTS[DATA_SOURCE]
LEGACY_PATH = LEGACY_GRAPH_PATHS.get(DATA_SOURCE)
if LEGACY_PATH and LEGACY_PATH.exists() and not GRAPH_OUTPUT_PATH.exists():
    print(f"Legacy graph detected at {LEGACY_PATH}. New exports will use {GRAPH_OUTPUT_PATH}.")
print(f"Using data source: {DATA_SOURCE}")
print(f"Graph exports will be written to: {GRAPH_OUTPUT_PATH}")

Using data source: sbb
Graph exports will be written to: datasets/switzerland/sbb_rail_network.gpickle


In [133]:
if DATA_SOURCE == "sbb":
    sbb_line_data = pd.read_csv(SBB_LINE_PATH, sep=';')
    sbb_stations_data = pd.read_csv(SBB_STATION_PATH, sep=';')
    sbb_line_and_station_data = sbb_line_data.merge(
        sbb_stations_data,
        left_on="Didok number",
        right_on="number",
        how="left",
        suffixes=("", "_didok"),
    )

    print(f"Line table shape: {sbb_line_data.shape}")
    print(f"Stations table shape: {sbb_stations_data.shape}")
    print("Combined columns:")
    print(sorted(sbb_line_and_station_data.columns)[:10], "...")
else:
    net_segments = gpd.read_file(SWISSTOPO_GDB_PATH, layer='Netzsegment')
    net_nodes = gpd.read_file(SWISSTOPO_GDB_PATH, layer='Netzknoten')

    print(f"Segments table shape: {net_segments.shape}")
    print(f"Nodes table shape: {net_nodes.shape}")
    print("Segment columns:")
    print(sorted(net_segments.columns))
    print("Node columns:")
    print(sorted(net_nodes.columns))

Line table shape: (1882, 11)
Stations table shape: (56244, 47)
Combined columns:
['Didok number', 'Geopos', 'Geopos_didok', 'KM', 'Line', 'Line.1', 'OPUIC', 'Station abbreviation', 'Stop name', 'Stop name.1'] ...


In [134]:
def parse_geopos(value):
    if isinstance(value, str) and ',' in value:
        lat_str, lon_str = value.split(',', 1)
        try:
            lat = float(lat_str.strip())
            lon = float(lon_str.strip())
        except ValueError:
            return None
        if -90 <= lat <= 90 and -180 <= lon <= 180:
            return lat, lon
    return None


def classify_station(abbreviation, fallback_rows=None):
    abbr = (abbreviation or "").strip().upper()
    if abbr and abbr in station_abbreviation_set:
        return True
    fallback_rows = fallback_rows or []
    for row in fallback_rows:
        row_abbr = row.get('Station abbreviation')
        if isinstance(row_abbr, str) and row_abbr.strip().upper() in station_abbreviation_set:
            return True
    return False


def flatten_lines(geom):
    if geom is None or geom.is_empty:
        return []
    coords = []
    if geom.geom_type == 'LineString':
        coords.extend((pt[1], pt[0]) for pt in geom.coords)
    elif geom.geom_type == 'MultiLineString':
        for line in geom.geoms:
            coords.extend((pt[1], pt[0]) for pt in line.coords)
    return coords


G = nx.Graph()
node_records = []
positions = {}

if DATA_SOURCE == "sbb":
    LINE_COL = "Line"
    STATION_COL = "Station abbreviation"
    ORDER_COL = "KM"

    node_groups = sbb_line_and_station_data.dropna(subset=[STATION_COL]).groupby(STATION_COL)
    for station, group in node_groups:
        row_dicts = group.to_dict('records')
        coords = None
        for row in row_dicts:
            coords = parse_geopos(row.get('Geopos')) or parse_geopos(row.get('Geopos_didok'))
            if coords:
                break
        lat, lon = coords if coords else (None, None)
        label = next((row.get('Stop name') for row in row_dicts if isinstance(row.get('Stop name'), str) and row.get('Stop name')), station)
        is_station = classify_station(station, row_dicts)
        node_attrs = {
            'label': label,
            'abbreviation': station,
            'lat': lat,
            'lon': lon,
            'is_station': is_station,
            'rows': row_dicts,
            'source': 'sbb',
        }
        G.add_node(station, **node_attrs)
        positions[station] = (lat, lon)
        node_records.append({
            'node_id': station,
            'label': label,
            'lat': lat,
            'lon': lon,
            'is_station': is_station,
            'source': 'sbb',
        })

    ordered_df = (
        sbb_line_and_station_data
        .dropna(subset=[LINE_COL, STATION_COL, ORDER_COL])
        .sort_values([LINE_COL, ORDER_COL])
    )

    for line_id, group in ordered_df.groupby(LINE_COL):
        stops = group[STATION_COL].tolist()
        kms = group[ORDER_COL].tolist()
        row_dicts = group.to_dict('records')

        for idx, (u, v) in enumerate(zip(stops[:-1], stops[1:])):
            segment_meta = {
                'line_id': line_id,
                'order_index': idx,
                'from_station': u,
                'to_station': v,
                'from_km': kms[idx],
                'to_km': kms[idx + 1],
                'from_row': row_dicts[idx],
                'to_row': row_dicts[idx + 1],
            }
            if G.has_edge(u, v):
                G[u][v]['lines'].add(line_id)
                G[u][v]['segments'].append(segment_meta)
            else:
                G.add_edge(
                    u,
                    v,
                    lines={line_id},
                    segments=[segment_meta],
                    source='sbb',
                )

    for u, v, data in G.edges(data=True):
        data['lines'] = sorted(data['lines'])
else:
    nodes_gdf = net_nodes.to_crs(4326)
    segments_gdf = net_segments
    segments_wgs84 = segments_gdf.to_crs(4326)

    for _, row in nodes_gdf.iterrows():
        node_id = row['xtf_id']
        abbr = row.get('Betriebspunkt_Abkuerzung')
        label = abbr or row.get('Betriebspunkt_Name') or node_id
        lat = row.geometry.y
        lon = row.geometry.x
        is_station = False
        if isinstance(abbr, str) and abbr.strip():
            is_station = abbr.strip().upper() in station_abbreviation_set
        node_attrs = {
            'label': label,
            'abbreviation': abbr,
            'lat': lat,
            'lon': lon,
            'is_station': is_station,
            'rows': [row.drop(labels='geometry').to_dict()],
            'source': 'swisstopo',
        }
        G.add_node(node_id, **node_attrs)
        positions[node_id] = (lat, lon)
        node_records.append({
            'node_id': node_id,
            'label': label,
            'lat': lat,
            'lon': lon,
            'is_station': is_station,
            'source': 'swisstopo',
        })

    for _, row in segments_wgs84.iterrows():
        u = row['rAnfangsknoten']
        v = row['rEndknoten']
        if pd.isna(u) or pd.isna(v):
            continue
        if u not in G.nodes or v not in G.nodes:
            continue
        lines = [row['Name']] if isinstance(row.get('Name'), str) else []
        segment_meta = {
            'segment_id': row['xtf_id'],
            'line_name': row.get('Name'),
            'track_count': row.get('AnzahlStreckengleise'),
            'gauge': row.get('Spurweite'),
            'electrified': row.get('Elektrifizierung'),
            'coords_wgs84': flatten_lines(row.geometry),
        }
        if G.has_edge(u, v):
            combined = set(G[u][v]['lines'])
            combined.update(lines)
            G[u][v]['lines'] = sorted(combined)
            G[u][v]['segments'].append(segment_meta)
        else:
            G.add_edge(
                u,
                v,
                lines=sorted(lines),
                segments=[segment_meta],
                source='swisstopo',
            )

node_summary_df = pd.DataFrame(node_records)
print(f"Nodes prepared with coordinates: {len(node_summary_df)}")

Nodes prepared with coordinates: 1355


In [135]:
print(f"Nodes: {G.number_of_nodes():,}")
print(f"Edges: {G.number_of_edges():,}")
print(f"Connected components: {nx.number_connected_components(G):,}")

Nodes: 1,355
Edges: 1,512
Connected components: 10


In [136]:
components = list(nx.connected_components(G))
rows = []
for nodeset in components:
    sub = G.subgraph(nodeset)
    rows.append({
        'num_nodes': sub.number_of_nodes(),
        'num_edges': sub.number_of_edges(),
    })
cc_df = pd.DataFrame(rows)
cc_df = cc_df.sort_values('num_nodes', ascending=False).reset_index(drop=True)
cc_df.insert(0, 'rank', cc_df.index + 1)
cc_df

,rank,num_nodes,num_edges
0,1,1338,1504
1,2,2,1
2,3,2,1
3,4,2,1
4,5,2,1
5,6,2,1
6,7,2,1
7,8,2,1
8,9,2,1
9,10,1,0


In [137]:
components_by_size = sorted(components, key=len, reverse=True)
if len(components_by_size) > 1:
    small_components = components_by_size[1:]
else:
    small_components = []

rows = []
component_meta = {}
for rank, nodeset in enumerate(small_components, start=2):
    sub = G.subgraph(nodeset)
    component_meta[rank] = {
        'num_nodes': sub.number_of_nodes(),
        'num_edges': sub.number_of_edges(),
    }
    for node_id in sorted(nodeset):
        data = G.nodes[node_id]
        rows.append({
            'component_rank': rank,
            'node_id': node_id,
            'label': data.get('label', node_id),
            'abbreviation': data.get('abbreviation'),
            'is_station': data.get('is_station', False),
            'source': data.get('source', DATA_SOURCE),
        })

small_cc_df = (
    pd.DataFrame(rows)
    .sort_values(['component_rank', 'label'])
    .reset_index(drop=True)
)

small_cc_df

,component_rank,node_id,label,abbreviation,is_station,source
0,2,ASKO,Amsteg Kabelstollen Ost,ASKO,False,sbb
1,2,ASSW,Amsteg Kabelstollen West,ASSW,False,sbb
2,3,ASZO,Amsteg Zugangsstollen Ost,ASZO,False,sbb
3,3,ASZW,Amsteg Zugangsstollen West,ASZW,False,sbb
4,4,BOAN,Bodio cunicolo di aggira. nord,BOAN,False,sbb
5,4,BOCS,Bodio cunicolo di aggira. sud,BOCS,False,sbb
6,5,BOZN,Bozberg Zugangsstollen Nord,BOZN,False,sbb
7,5,BOZS,Bozberg Zugangsstollen Sud,BOZS,False,sbb
8,6,FACE,Faido cunicolo di acc. est,FACE,False,sbb
9,6,FACO,Faido cunicolo di acc. ovest,FACO,False,sbb


In [138]:
pair_rows = []
singleton_rows = []

if small_cc_df.empty:
    print("No satellite components detected outside the largest component.")
else:
    for comp_rank, group in small_cc_df.groupby('component_rank'):
        group = group.sort_values('label')
        meta = component_meta.get(comp_rank, {'num_nodes': len(group), 'num_edges': max(len(group) - 1, 0)})
        if len(group) < 2:
            singleton_rows.append({
                'component_rank': comp_rank,
                'num_nodes': meta['num_nodes'],
                'num_edges': meta['num_edges'],
                'node_id': group.iloc[0]['node_id'],
                'label': group.iloc[0]['label'],
            })
            continue
        a, b = group.iloc[0], group.iloc[1]
        pair_rows.append({
            'component_rank': comp_rank,
            'num_nodes': meta['num_nodes'],
            'num_edges': meta['num_edges'],
            'node1': f"{a['node_id']} ({a['label']})",
            'node2': f"{b['node_id']} ({b['label']})",
        })

small_pairs_df = (
    pd.DataFrame(pair_rows)
    .sort_values('component_rank')
    .reset_index(drop=True)
)

singleton_components_df = (
    pd.DataFrame(singleton_rows)
    .sort_values('component_rank')
    .reset_index(drop=True)
)

print("Pair summary:")
print(small_pairs_df)
print("Singleton components:")
print(singleton_components_df if not singleton_components_df.empty else "None")

Pair summary:
   component_rank  num_nodes  num_edges  \
0               2          2          1   
1               3          2          1   
2               4          2          1   
3               5          2          1   
4               6          2          1   
5               7          2          1   
6               8          2          1   
7               9          2          1   

                                   node1                                 node2  
0         ASKO (Amsteg Kabelstollen Ost)       ASSW (Amsteg Kabelstollen West)  
1       ASZO (Amsteg Zugangsstollen Ost)     ASZW (Amsteg Zugangsstollen West)  
2  BOAN (Bodio cunicolo di aggira. nord)  BOCS (Bodio cunicolo di aggira. sud)  
3     BOZN (Bozberg Zugangsstollen Nord)     BOZS (Bozberg Zugangsstollen Sud)  
4      FACE (Faido cunicolo di acc. est)   FACO (Faido cunicolo di acc. ovest)  
5                  LNTW (Heimbach (Spw))             LNTO (Luzern Tiefbahnhof)  
6       SETS (Sedrun Entluftung

In [139]:
if components_by_size:
    largest_nodes = components_by_size[0]
    print(f"Largest component nodes: {len(largest_nodes):,}")
    print(f"Largest component share: {len(largest_nodes) / G.number_of_nodes():.2%}")
else:
    print("Graph has no components.")

Largest component nodes: 1,338
Largest component share: 98.75%


In [140]:
if EXPORT_GRAPH:
    GRAPH_OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    with GRAPH_OUTPUT_PATH.open('wb') as f:
        pickle.dump(G, f)
    print(f"Serialized graph to {GRAPH_OUTPUT_PATH}")
else:
    print("Skipping graph export.")

Serialized graph to datasets/switzerland/sbb_rail_network.gpickle
